In [32]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import time
import pickle

In [33]:
header = ({'User-Agent': 
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 ' + \
            '(KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

first_page = "https://www.tripadvisor.com.au/Attractions-g4-Activities-Europe.html"
response = get(first_page, headers=header)

#If response is 200, we have success
print(response)

<Response [200]>


In [5]:
#Get urls of the pages listing the places
first_page_html = BeautifulSoup(response.text, 'html.parser')
length = int(first_page_html.find_all('a', class_="pageNum taLnk")[-1].text)

list_urls = [first_page] + ["https://www.tripadvisor.com.au/Attractions-g4-Activities-oa" + str(50*i - 80) + "-Europe.html" \
                            for i in range(length+1)]
print(length, len(list_urls))

972

In [6]:
%%time
# WARNING TAKES 30 MINS TO RUN [36]
#Get urls of each place
place_urls = []
i=0
for list_page in list_urls:
    response = get(list_page, headers=header)
    page_html = BeautifulSoup(response.text, 'html.parser')
    urls = page_html.find_all('a')
    urls = [url.get('href') for url in urls]
    urls = set([url for url in urls if ('/Attractions' in url) and ('Europe.' not in url)])
    place_urls += ['https://www.tripadvisor.com.au' + url for url in urls]
    
    time.sleep(0.05)
    if (i%100==0) & (i!=0):
        print(i)
    i += 1
    
#Save list
with open('place_urls.txt', 'wb') as f:
    pickle.dump(place_urls, f)
print('number of pages = ' + str(i) + ', number of places =', len(place_urls))

100
200
300
400
500
600
700
800
900
number of pages = 972 , number of places =  48497
Wall time: 36min 3s


In [34]:
with open('place_urls.txt', 'rb') as f:
    place_urls = pickle.load(f)

In [36]:
%%time
attraction_urls = []

for place_page in place_urls[::3000]:
    place = ' '.join(place_page.split('-')[-1].split('.')[0].split('_'))
    response = get(place_page, headers=header)
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    #Big places 
    big_place = False
    if 'Browse by Category' in str(page_html):
        big_place = True
        
    category_urls = []
    
    if big_place:
        category_class = "attractions-attraction-overview-main-ShelfCarouselItem__item_wrapper--3pJJS "
        category_containers = page_html.find_all('div', class_=category_class)
        for category in category_containers:
            category_urls.append('https://www.tripadvisor.com.au' + category.find_all('a')[0].get('href'))
        
        for category_page in category_urls:
            response = get(category_page, headers=header)
            page_html = BeautifulSoup(response.text, 'html.parser')
            attraction_containers = page_html.find_all('div', class_='attraction_element')
            for attraction in attraction_containers:
                try:
                    attraction_urls.append('https://www.tripadvisor.com.au' + \
                                           attraction.find_all('div', class_='listing_info')[0].find_all('a')[0].get('href'))
                except:
                    pass
        
    else:
        
        
    time.sleep(0.05)
    
attraction_urls = list(set(attraction_urls))
print(attraction_urls[:20:20])

['https://www.tripadvisor.com.au/Attraction_Review-g298507-d1547134-Reviews-Divo_Ostrov-St_Petersburg_Northwestern_District.html', 'https://www.tripadvisor.com.au/Attraction_Review-g298507-d12703485-Reviews-Volshebnaya_Milya-St_Petersburg_Northwestern_District.html', 'https://www.tripadvisor.com.au/Attraction_Review-g298507-d4931477-Reviews-Voda_AquaClub_and_Hotel-St_Petersburg_Northwestern_District.html', 'https://www.tripadvisor.com.au/Attraction_Review-g298507-d8714364-Reviews-Adventure_Park_Tree_To_Tree-St_Petersburg_Northwestern_District.html', 'https://www.tripadvisor.com.au/Attraction_Review-g298507-d7296388-Reviews-Rock_Town-St_Petersburg_Northwestern_District.html', 'https://www.tripadvisor.com.au/Attraction_Review-g298507-d6502283-Reviews-Gagarin_Amusement_Park-St_Petersburg_Northwestern_District.html', 'https://www.tripadvisor.com.au/Attraction_Review-g2394587-d8545197-Reviews-Kingwinch-Kronshtadt_Kronshtadtsky_District_St_Petersburg_Northwestern_District.html', 'https://www

In [39]:
len(attraction_urls)

993

In [27]:
place_page = place_urls[0]
place = ' '.join(place_page.split('-')[-1].split('.')[0].split('_'))
response = get(place_page, headers=header)
page_html = BeautifulSoup(response.text, 'html.parser')

category_class = "attractions-attraction-overview-main-ShelfCarouselItem__item_wrapper--3pJJS "
category_containers = page_html.find_all('div', class_=category_class)
print(place, len(category_containers), place_page)
category_page = 'https://www.tripadvisor.com.au' + category_containers[0].find_all('a')[0].get('href') 
response = get(category_page, headers=header)
page_html = BeautifulSoup(response.text, 'html.parser')
attraction_containers = page_html.find_all('div', class_='attraction_element')
attraction_containers[0].find_all('div', class_='listing_info')[0].find_all('a')[0].get('href')

St Petersburg Northwestern District 73 https://www.tripadvisor.com.au/Attractions-g298507-Activities-St_Petersburg_Northwestern_District.html


'/Attraction_Review-g298507-d1547134-Reviews-Divo_Ostrov-St_Petersburg_Northwestern_District.html'

In [5]:
(1, 2, 3,)

(1, 2, 3)

In [7]:
(1,)

1